ST-GCN -> redifined for Exercise

In [1]:
#@title Set up
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/03_assignment/04_HRI/STGCN-SWMV


%cd torchlight 
!python setup.py install 
%cd ..


#!bash tools/rsc/get_data.sh
#!bash tools/rsc/get_weights.sh

## test for UOW dataset
#!python main.py stgcn_swmv --dataset=UOW --use_gpu=True -c config/stgcn_swmv/UOW/test.yaml

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/03_assignment/04_HRI/STGCN-SWMV


### Data Format:
(N C V T M)

- N: Number of input action sequences
- C: Channels(default=3)
- V: Size of sliding windows (defalut=50)
- T: Amount of Joints
- M: Step size(the length of  the sequence that moves between each window)(default = 1)


### MM-FIT dataset format: 17 joints
wxx_labels.csv: [start,end,rep,action] eg:22147	22990	10	lunges
1. 'hip'
2. 'left_hip'
3. 'left_knee'
4. 'left_foot'
5. 'right_hip'
6. 'right_knee'
7. 'right_foot'
8. 'spine'
9. 'thorax'
10. 'nose'
11. 'head'
12. 'right_shoulder'
13. 'right_elbow'
14. 'right_wrist'
15. 'left_shoulder'
16. 'left_elbow
17. left_wrist' 
### NTURGB+D Dataset: 25 joints
- 1-base of the spine 
- 2-middle of the spine
- 3-neck 
- 4-head 
- 5-left shoulder 
- 6-left elbow 
- 7-left wrist 
- 8-left hand
- 9-right shoulder 
- 10-right elbow 
- 11-right wrist 
- 12-right hand 
- 13-left hip 
- 14-left knee 
- 15-left ankle 
- 16-left foot 
- 17-right hip 
- 18-right knee 
- 19-right ankle 
- 20-right foot 
- 21-spine 
- 22-tip of the left hand 
- 23-left thumb 
- 24-tip of the right hand 
- 25-right thumb
![rsc/anatomy.png]("./rsc/anatomy.png")

In our work, we utilize the spatial temporal graph to form hierarchical representation of the skeleton sequences. 

 we construct an undirected spatial temporal graph $G = (V, E)$ on a skeleton sequence with $N$ joints and $T$ frames featuring
both intra-body and inter-frame connection.

In [6]:
import numpy as np
import csv
import os
import pickle
from tools import pose_embedding
from tqdm import trange,tqdm

In [7]:
PATH = "/content/drive/MyDrive/03_assignment/04_HRI/mm-fit-master/mm-fit"
ACTIONS = ['squats', 'lunges', 'bicep_curls', 'situps', 'pushups', 'tricep_extensions', 'dumbbell_rows',
           'jumping_jacks', 'dumbbell_shoulder_press', 'lateral_shoulder_raises', 'non_activity']
TRAIN_W_IDs = ['01', '02', '03', '04', '06', '07', '08', '16', '17', '18']
VAL_W_IDs = ['14', '15', '19']
TEST_W_IDs = ['00', '05', '12', '13', '20']
# if args.unseen_test_set:
#     TEST_W_IDs = ['00', '05', '12', '13', '20']
# else:
#     TEST_W_IDs = ['09', '10', '11']

In [8]:
!pip install mediapipe

In [9]:
def load_labels(filepath):
    """
    Loads and reads CSV MM-Fit CSV label file.
    :param filepath: File path to a MM-Fit CSV label file.
    :return: List of lists containing label data, (Start Frame, End Frame, Repetition Count, Activity) for each
    exercise set.
    """
    labels = []
    with open(filepath, 'r') as csv_file:
        reader = csv.reader(csv_file)
        for line in reader:
            labels.append([int(line[0]), int(line[1]), int(line[2]), line[3]]) ## [start,end,rep,action] 
    return labels
def get_subset(data, start=0, end=None):
    """
    Returns a subset of modality data.
    :param data: Modality (numpy array).
    :param start: Start frame of subset.
    :param end: End frame of subset.
    :return: Subset of data (numpy array).
    """
    if data is None:
        return None

    # Pose data
    if len(data.shape) == 3:
        if end is None:
            end = data[0, -1, 0]
        return data[:, np.where(((data[0, :, 0]) >= start) & ((data[0, :, 0]) <= end))[0], :]

    # Accelerometer, gyroscope, magnetometer and heart-rate data
    else:
        if end is None:
            end = data[-1, 0]
        return data[np.where((data[:, 0] >= start) & (data[:, 0] <= end)), :][0]
def load_modality(filepath):
    """
    Loads modality from filepath and returns numpy array, or None if no file is found.
    :param filepath: File path to MM-Fit modality.
    :return: MM-Fit modality (numpy array).
    """
    try:
        mod = np.load(filepath)
    except FileNotFoundError as e:
        mod = None
        print('{}. Returning None'.format(e))
    return mod

In [16]:
t = np.load("/content/drive/MyDrive/03_assignment/04_HRI/mm-fit-master/mm-fit/w00/w00_pose_3d.npy")
t.shape

(3, 63918, 18)

In [21]:
t[0,1,0]

4051.0

In [48]:
np.array_split(t,10,1)[0][:,:,:,np.newaxis].shape

(3, 6392, 18, 1)

In [22]:
embedder = pose_embedding.FullBodyPoseEmbedder(train_flag=True)
ACTIONS = {'squats': 0, 'lunges': 1, 'bicep_curls': 2, 'situps': 3, 'pushups': 4, 'tricep_extensions': 5,
                        'dumbbell_rows': 6, 'jumping_jacks': 7, 'dumbbell_shoulder_press': 8,
                        'lateral_shoulder_raises': 9, 'non_activity': 10}

skeleton_window_length = 50
def data_preprocess(W_IDs=[],skeleton_window_length = 50,path=None,types="train"):
  ## output
  LABELS= ([],[],[])
  DATA = []
  for _ in trange(len(W_IDs),desc='1st loop'):
    for w_id in W_IDs:
      workout_path = os.path.join(PATH,'w'+w_id)
      files = os.listdir(workout_path)
      label_path = None
      labels = []
      modalities = []
      
      
      for file in files:
        if 'labels' in file:
          label_path = os.path.join(workout_path, file)
          labels = load_labels(label_path)
          continue
        if 'pose_3d' in file:
          modality_path = os.path.join(workout_path, file)
          modalities = load_modality(modality_path) # (3, 47997, 18) -> (channels, frams, joints)
                        
      if label_path is None:
        raise Exception('Error: Label file not found for workout {}.'.format(w_id))

      if modalities is None:
        modalities = np.zeros(3,skeleton_window_length,17)
        completion = 0.0
        label = 'non_activity'
        # label = 'non_activity'
        # reps = 0
        # for row in labels:
        #   label = row[3]
        #   reps = row[2]
        #   agg = get_subset(modadities, start=row[0], end=row[1])
        #   li = np.array_split(agg, reps, 1)
        #   frame_limit = li[-1].shape[1]
        #   for arr in li:
        #     arr[:]
        #     arr[:,:frame_limit,1:,np.newaxis]
        #     LABELS[0].append(label)
        #     LABELS[1].append(ACTIONS[label])
      else:
          for i in tqdm(range(modalities.shape[1]),desc='2nd loop'):
            frame = modalities[0,i,0]
            final_frame = modalities[0,-1,0]
            end_frames = None
            label = 'non_activity'
            reps = 0
            completion = 1.0
            
            for row in labels:
              if (frame > (row[0] - skeleton_window_length/2)) and (frame < (row[1] + skeleton_window_length/2)):
                label = row[3]
                reps = row[2]
                final_frame = row[1]
                end_frames = np.round(np.linspace(0,1,final_frame+1,dtype='float32')*final_frame)[1:]+1
                break
            if end_frames is None:
              continue
            arr = modalities[:, i:(i+skeleton_window_length), :]
            if arr.shape[1]<skeleton_window_length:
              break
            seq = []
            completion = 0.0
            for j in range(arr.shape[1]):
              if arr[0,j,0] > final_frame:
                completion = 0.0
              else:
                completion = arr[0,j,0]/end_frames #float
                completion = completion[np.where(completion<=1.0)[0]].max()  ## How far the action is completed
              seq.append(embedder(arr[:,j,1:].T).T)
            DATA.append(seq) ## consider coordinates only
            LABELS[0].append(label)
            LABELS[1].append(ACTIONS[label])
            LABELS[2].append(completion)

    np.save("{}/mmfit_{}_data.npy".format(path,types),np.array(DATA,dtype='float32')[:,:,:,:,np.newaxis].transpose(0,2,1,3,4))
    pickle.dump(LABELS, open('{}/mmfit_{}_label.pkl'.format(path,types), 'wb'))
    
    return np.array(DATA,dtype='float32')[:,:,:,:,np.newaxis].transpose(0,2,1,3,4),LABELS
  

In [23]:
#@title Example for PRE-process
data_test,lab_test = data_preprocess(TEST_W_IDs,path="data/MMFIT",types="train")

2nd loop: 100%|██████████| 63918/63918 [01:19<00:00, 804.67it/s] 

2nd loop: 100%|██████████| 57157/57157 [01:15<00:00, 759.96it/s]

2nd loop: 100%|██████████| 67787/67787 [01:17<00:00, 870.67it/s]

2nd loop: 100%|██████████| 68561/68561 [01:32<00:00, 739.10it/s]

1st loop:   0%|          | 0/5 [08:15<?, ?it/s]


## Train ST-GCN Model for MMFIT Dataset

Due to the limited source, we trained 50 epoch(batch size = 128), and selected the 48th epoch as the final model
```
[04.23.23|21:54:24] Training epoch: 45
/content/drive/MyDrive/03_assignment/04_HRI/STGCN-SWMV/tools/focal_loss.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  logpt = F.log_softmax(input)
[04.23.23|21:54:27] 	Iter 28700 Done. | loss: 0.0012 | lr: 0.1000000000
[04.23.23|21:54:32] 	Iter 28800 Done. | loss: 0.0085 | lr: 0.1000000000
[04.23.23|21:54:36] 	Iter 28900 Done. | loss: 0.0001 | lr: 0.1000000000
[04.23.23|21:54:40] 	Iter 29000 Done. | loss: 0.0011 | lr: 0.1000000000
[04.23.23|21:54:45] 	Iter 29100 Done. | loss: 0.0014 | lr: 0.1000000000
[04.23.23|21:54:49] 	Iter 29200 Done. | loss: 0.0012 | lr: 0.1000000000
[04.23.23|21:54:51] 	mean_loss: 0.0032134797350082873
[04.23.23|21:54:51] Time consumption:
[04.23.23|21:54:51] Done.
[04.23.23|21:54:51] The model has been saved as ./work_dir/MMFIT//epoch46_model.pt.
[04.23.23|21:54:51] Eval epoch: 45
[04.23.23|22:01:17] 	mean_loss: 0.47524454922881637
/content/drive/MyDrive/03_assignment/04_HRI/STGCN-SWMV/tools/recognition.py:96: RuntimeWarning: invalid value encountered in true_divide
  acc = intersection / (sum(cm))
/content/drive/MyDrive/03_assignment/04_HRI/STGCN-SWMV/tools/recognition.py:102: RuntimeWarning: invalid value encountered in true_divide
  IoU = intersection / union
Non normalized confusion matrix :
 [[3191   30   39    0    0   29    0  135    0    0]
 [1428 6971 2540    0    0    1  170   80    8  443]
 [ 107  117 3827    0    0    0   44    2    0    0]
 [   0  317    0 7463    0    0    0    0    0    0]
 [   0    0    0    0 5455    0    0    0    0    0]
 [   0    0    0    0    0 5813    0    0    0    0]
 [ 711    1    0    0    0   14 4803    7    0   24]
 [  77  477   16    0    0    0    0 1904    0    0]
 [   0    0    0    0    0    0    0    0 6943  109]
 [   0    0    0    0    0   30    0    0  271 4507]]
Figure(2300x1800)

Groundtruth labels:  [0 0 0 ... 9 9 9]

Predicted labels:  [1 1 1 ... 9 9 9]
[04.23.23|22:01:18] Accuracy: 87.56%
[04.23.23|22:01:18] F1-Score: 87.63%
```

In [10]:
!python main.py stgcn_swmv --dataset=MMFIT --use_gpu=True -c config/stgcn_swmv/MMFIT/train.yaml

[04.23.23|16:40:00] Parameters:
{'work_dir': './work_dir/MMFIT/', 'config': 'config/stgcn_swmv/MMFIT/train.yaml', 'phase': 'train', 'save_result': False, 'start_epoch': 0, 'num_epoch': 50, 'use_gpu': True, 'device': [0], 'log_interval': 100, 'save_interval': 1, 'eval_interval': 1, 'save_log': True, 'print_log': True, 'pavi_log': False, 'loader': 'tools.loader.Loader', 'num_worker': 4, 'train_loader_args': {'data_path': './data/MMFIT/mmfit_train_data.npy', 'label_path': './data/MMFIT/mmfit_train_label.pkl', 'debug': False}, 'test_loader_args': {'data_path': './data/MMFIT/mmfit_val_data.npy', 'label_path': './data/MMFIT/mmfit_val_label.pkl'}, 'batch_size': 128, 'test_batch_size': 1, 'debug': False, 'model': 'network.stgcn_swmv.Model', 'model_args': {'in_channels': 3, 'num_class': 11, 'dropout': 0.5, 'edge_importance_weighting': True, 'graph_args': {'layout': 'skeleton_layout', 'strategy': 'spatial'}}, 'weights': None, 'ignore_weights': [], 'dataset': 'MMFIT', 'base_lr': 0.1, 'step': [100

In [12]:
## test for UOW dataset
!python main.py stgcn_swmv --dataset=MMFIT --use_gpu=True -c config/stgcn_swmv/MMFIT/test.yaml

[04.23.23|22:40:03] Loading STGCN-SWMV weights [A].
[04.23.23|22:40:03] Loading STGCN-SWMV weights [data_bn.weight].
[04.23.23|22:40:03] Loading STGCN-SWMV weights [data_bn.bias].
[04.23.23|22:40:03] Loading STGCN-SWMV weights [data_bn.running_mean].
[04.23.23|22:40:03] Loading STGCN-SWMV weights [data_bn.running_var].
[04.23.23|22:40:03] Loading STGCN-SWMV weights [data_bn.num_batches_tracked].
[04.23.23|22:40:03] Loading STGCN-SWMV weights [st_gcn_networks.0.gcn.conv.weight].
[04.23.23|22:40:03] Loading STGCN-SWMV weights [st_gcn_networks.0.gcn.conv.bias].
[04.23.23|22:40:03] Loading STGCN-SWMV weights [st_gcn_networks.0.tcn.0.weight].
[04.23.23|22:40:03] Loading STGCN-SWMV weights [st_gcn_networks.0.tcn.0.bias].
[04.23.23|22:40:03] Loading STGCN-SWMV weights [st_gcn_networks.0.tcn.0.running_mean].
[04.23.23|22:40:03] Loading STGCN-SWMV weights [st_gcn_networks.0.tcn.0.running_var].
[04.23.23|22:40:03] Loading STGCN-SWMV weights [st_gcn_networks.0.tcn.0.num_batches_tracked].
[04.23.2